In [58]:
import requests
from bs4 import BeautifulSoup as bs4
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [4]:
base_url = 'https://rwandahouseland.com/forRent'
response = requests.get(base_url)
response.status_code

200

In [10]:
soup = bs4(response.text, 'html.parser')
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<title>RwandaHouseland - Find Your Dream Home</title>
<link href="https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css" rel="stylesheet"/>
<link href="https://cdnjs.cloudflare.com/ajax/libs/boxicons/2.1.4/css/boxicons.min.css" rel="stylesheet"/>
<link href="https://rwandahouseland.com/images/rogo1.jpg" rel="icon" type="image/icon type"/>
<style>
        /* Background Animation */
        @keyframes slideBackground {
            0%, 25% { background-image: url('/img/log3.jpg'); }
            33%, 58% { background-image: url('/img/log4.jpg'); }
            66%, 91% { background-image: url('/img/log5.jpg'); }
            100% { background-image: url('/img/log6.jpg'); }
        }
        .sliding-bg {
            background-size: cover;
            background-position: center;
            animation: slideBackground 20s infinite;
         

In [19]:
property_card = soup.find_all('div', class_='p-3 space-y-2')
property_card

[<div class="p-3 space-y-2">
 <h5 class="text-sm font-medium text-gray-900 truncate">
 <a class="hover:text-green-600 transition" href="https://rwandahouseland.com/property_view/1059">
                             Spacious &amp;modern 4bedrooms house for rent in kibagabaga
                         </a>
 </h5>
 <div class="space-y-1.5">
 <!-- Price -->
 <div class="flex items-center gap-2">
 <svg class="w-4 h-4 text-green-600 shrink-0" fill="none" stroke="currentColor" viewbox="0 0 24 24" xmlns="http://www.w3.org/2000/svg">
 <path d="M12 8c-1.657 0-3 .895-3 2s1.343 2 3 2 3 .895 3 2-1.343 2-3 2m0-8c1.11 0 2.08.402 2.599 1M12 8V7m0 1v8m0 0v1m0-1c-1.11 0-2.08-.402-2.599-1M21 12a9 9 0 11-18 0 9 9 0 0118 0z" stroke-linecap="round" stroke-linejoin="round" stroke-width="2"></path>
 </svg>
 <span class="text-sm text-gray-900">3,000 USD</span>
 </div>
 <!-- Property Type -->
 <div class="flex items-center gap-2">
 <svg class="w-4 h-4 text-green-600 shrink-0" fill="none" stroke="currentColor" vie

In [42]:
data = []
# Loop through and extract data
for property_div in property_card:
    # Extract title
    title_tag = property_div.find('h5', class_='text-sm font-medium text-gray-900 truncate')
    title = title_tag.get_text(strip=True) if title_tag else "N/A"

    # Extract price
    price_tag = property_div.find('span', class_='text-sm text-gray-900')
    price = price_tag.get_text(strip=True) if price_tag else "N/A"

    # Extract all 'text-xs text-gray-600 truncate' spans (they appear multiple times)
    detail_spans = property_div.find_all('span', class_='text-xs text-gray-600 truncate')

    property_type = detail_spans[0].get_text(strip=True) if len(detail_spans) > 0 else "N/A"
    advert_type = detail_spans[1].get_text(strip=True) if len(detail_spans) > 1 else "N/A"
    location = detail_spans[2].get_text(strip=True) if len(detail_spans) > 2 else "N/A"

    data.append({
        'Title': title,
        'Price': price,
        'Property_type': property_type,
        'Advert_type': advert_type,
        'Location': location
    })

    # # Print results
    # print(f"Title: {title}")
    # print(f"Price: {price}")
    # print(f"Type: {property_type}")
    # print(f"Advert: {advert_type}")
    # print(f"Location: {location}")
    # print("-----")
# Create DataFrame
df = pd.DataFrame(data)

# Show the DataFrame
df

,Title,Price,Property_type,Advert_type,Location
0,Spacious &modern 4bedrooms house for rent in k...,"3,000 USD",house,Rent,Kibagabaga
1,Beautiful full furnished house for rent in goo...,"3,000 USD",house,Rent,Niboye
2,Full furnished house for rent in kinyinya with...,"5,000 USD",house,Rent,Kinyinya
3,,"3,500 USD",house,Rent,Kiyovu
4,Nyarutarama full furnished house for rent in g...,"2,000 USD",house,Rent,Nyarutarama
...,...,...,...,...,...
59,Un furnished house for rent in kibagabaga,800 USD,house,Rent,Kibagabaga
60,Full furnished apartment for rent in nyarutara...,700 USD,apartment,Rent,Nyarutarama
61,Nice unfurnished house for rent in Remera near...,"800,000 FRW",house,Rent,Remera
62,belle maison à louer à gacuriro près des super...,"2,000 USD",house,Rent,Gacuriro


In [ ]:
def scrape_property(base_url, num_pages):

    all_products_data = []

    for page_num in range(1, num_pages + 1):
        if page_num == 1:
            page_url = base_url
        else:
            page_url = f"{base_url}page/{page_num}/"


        print(f"Scraping page: {page_url}")

        response = requests.get(page_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        property_card = soup.find_all('div', class_='p-3 space-y-2')

        data = []
        # Loop through and extract data
        for property_div in property_card:
            # Extract title
            title_tag = property_div.find('h5', class_='text-sm font-medium text-gray-900 truncate')
            title = title_tag.get_text(strip=True) if title_tag else "N/A"
        
            # Extract price
            price_tag = property_div.find('span', class_='text-sm text-gray-900')
            price = price_tag.get_text(strip=True) if price_tag else "N/A"
        
            # Extract all 'text-xs text-gray-600 truncate' spans (they appear multiple times)
            detail_spans = property_div.find_all('span', class_='text-xs text-gray-600 truncate')
        
            property_type = detail_spans[0].get_text(strip=True) if len(detail_spans) > 0 else "N/A"
            advert_type = detail_spans[1].get_text(strip=True) if len(detail_spans) > 1 else "N/A"
            location = detail_spans[2].get_text(strip=True) if len(detail_spans) > 2 else "N/A"
        
            data.append({
                'Title': title,
                'Price': price,
                'Property_type': property_type,
                'Advert_type': advert_type,
                'Location': location
            })


In [43]:
def scrape_property(base_url, num_pages):
    all_products_data = []

    for page_num in range(1, num_pages + 1):
        if page_num == 1:
            page_url = base_url
        else:
            page_url = f"{base_url}?page={page_num}"

        print(f"Scraping page: {page_url}")

        response = requests.get(page_url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        property_cards = soup.find_all('div', class_='p-3 space-y-2')

        for property_div in property_cards:
            title_tag = property_div.find('h5', class_='text-sm font-medium text-gray-900 truncate')
            title = title_tag.get_text(strip=True) if title_tag else "N/A"

            price_tag = property_div.find('span', class_='text-sm text-gray-900')
            price = price_tag.get_text(strip=True) if price_tag else "N/A"

            detail_spans = property_div.find_all('span', class_='text-xs text-gray-600 truncate')
            property_type = detail_spans[0].get_text(strip=True) if len(detail_spans) > 0 else "N/A"
            advert_type = detail_spans[1].get_text(strip=True) if len(detail_spans) > 1 else "N/A"
            location = detail_spans[2].get_text(strip=True) if len(detail_spans) > 2 else "N/A"

            all_products_data.append({
                'Title': title,
                'Price': price,
                'Property_type': property_type,
                'Advert_type': advert_type,
                'Location': location
            })

    # Convert all collected data into a DataFrame
    df = pd.DataFrame(all_products_data)
    return df

In [47]:
base_url = 'https://rwandahouseland.com/forRent'
num_pages = 8

df = scrape_property(base_url, num_pages)
df.head()

Scraping page: https://rwandahouseland.com/forRent
Scraping page: https://rwandahouseland.com/forRent?page=2
Scraping page: https://rwandahouseland.com/forRent?page=3
Scraping page: https://rwandahouseland.com/forRent?page=4
Scraping page: https://rwandahouseland.com/forRent?page=5
Scraping page: https://rwandahouseland.com/forRent?page=6
Scraping page: https://rwandahouseland.com/forRent?page=7
Scraping page: https://rwandahouseland.com/forRent?page=8


,Title,Price,Property_type,Advert_type,Location
0,Spacious &modern 4bedrooms house for rent in k...,"3,000 USD",house,Rent,Kibagabaga
1,Beautiful full furnished house for rent in goo...,"3,000 USD",house,Rent,Niboye
2,Full furnished house for rent in kinyinya with...,"5,000 USD",house,Rent,Kinyinya
3,,"3,500 USD",house,Rent,Kiyovu
4,Nyarutarama full furnished house for rent in g...,"2,000 USD",house,Rent,Nyarutarama


In [48]:
df

,Title,Price,Property_type,Advert_type,Location
0,Spacious &modern 4bedrooms house for rent in k...,"3,000 USD",house,Rent,Kibagabaga
1,Beautiful full furnished house for rent in goo...,"3,000 USD",house,Rent,Niboye
2,Full furnished house for rent in kinyinya with...,"5,000 USD",house,Rent,Kinyinya
3,,"3,500 USD",house,Rent,Kiyovu
4,Nyarutarama full furnished house for rent in g...,"2,000 USD",house,Rent,Nyarutarama
...,...,...,...,...,...
507,Full furnished apartment for rent in nyarutarama,900 USD,Apartment,Rent,
508,Furnished apartment for rent in kacyiru,650 USD,Apartment,Rent,
509,Unfurnished house for rent in Gisozi,800 USD,HOuse,Rent,
510,House,600 USD,Apartment,Rent,


In [75]:
# df.to_csv("https://rwandahouseland.com/forRent.csv", index=False)


In [60]:
# Data Cleaning and Preparation
# Function to convert prices to USD
rwf_to_usd = 0.000706184

def clean_and_convert_price(price_str):
    if isinstance(price_str, str):
        cleaned_price = re.sub(r'\s+', '', price_str).replace(',', '')
        if not cleaned_price:
            return np.nan
        elif 'USD' in cleaned_price:
            usd_value = cleaned_price.replace('USD', '').strip()
            if not usd_value:  # Check if anything is left after removing 'USD'
                return np.nan
            return float(usd_value)
        elif 'FRW' in cleaned_price:
            frw_value = cleaned_price.replace('FRW', '').strip()
            if not frw_value:  # Check if anything is left after removing 'FRW'
                return np.nan
            return float(frw_value) * rwf_to_usd
        elif cleaned_price.isdigit():
            return np.nan # Or your preferred handling for numbers without currency
        elif cleaned_price in ['USD', 'FRW', 'N/A']:
            return np.nan
        else:
            return np.nan
    elif pd.isna(price_str):
        return np.nan
    elif isinstance(price_str, (int, float)):
        return float(price_str)
    else:
        return np.nan

# Assuming your DataFrame is named 'df'
df['Price_USD'] = df['Price'].apply(clean_and_convert_price)

# Handle missing values in 'Title' (if any) - you can choose a different strategy
df['Title'].fillna('No Title Provided', inplace=True)

# Basic Descriptive Analysis

print("--- Basic Information ---")
print(df.info())

print("\n--- Descriptive Statistics for Price ---")
print(df['Price'].describe())

print("\n--- Value Counts for Categorical Columns ---")
for col in ['Property_type', 'Advert_type', 'Location']:
    print(f"\nValue counts for {col}:")
    print(df[col].value_counts())











--- Basic Information ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 512 entries, 0 to 511
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Title          512 non-null    object 
 1   Price          512 non-null    object 
 2   Property_type  512 non-null    object 
 3   Advert_type    512 non-null    object 
 4   Location       512 non-null    object 
 5   Price_USD      505 non-null    float64
dtypes: float64(1), object(5)
memory usage: 24.1+ KB
None

--- Descriptive Statistics for Price ---
count           512
unique           75
top       1,500 USD
freq             56
Name: Price, dtype: object

--- Value Counts for Categorical Columns ---

Value counts for Property_type:
Property_type
house        337
apartment    118
Apartment     26
House         18
offices        4
land           3
Plot/Land      3
HOuse          2
wirehouse      1
Name: count, dtype: int64

Value counts for Advert_type:
Advert_t

C:\Users\user\AppData\Local\Temp\ipykernel_976\495677398.py:37: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Title'].fillna('No Title Provided', inplace=True)


In [62]:
df.Price.unique()

array(['3,000 USD', '5,000 USD', '3,500 USD', '2,000 USD', '850,000 FRW',
       '2,500 USD', 'USD', '350,000,000 FRW', '1,600 USD', '600 USD',
       '700,000 FRW', '1,000 USD', '1,300 USD', '800 USD',
       '1,000,000 FRW', '500 USD', '1,500 USD', '600,000 FRW', '650 USD',
       'FRW', '1,200 USD', '2,200 USD', '800,000 FRW', '1,800 USD',
       '700 USD', '50 USD', '500,000 FRW', '4,000 USD', '400 USD',
       '400,000 FRW', '900 USD', 'N/A', '300 USD', '350,000 FRW',
       '1,200 FRW', '900,000 FRW', '1,700 USD', '500 FRW',
       '170,000,000 FRW', '1,500 FRW', '300,000 FRW', '20,000 USD',
       '2,500 FRW', '800 FRW', '450,000 FRW', '85,000 USD',
       '1,000,000 USD', '4,000,000 FRW', '2,300 FRW', '4 USD', '600 FRW',
       '2,000 FRW', '450,000,000 FRW', '1,000 FRW', '350 USD', '750 USD',
       '1,600 FRW', '65,000,000 FRW', '620 FRW', '25,000,000 FRW',
       '22,000,000 FRW', '250,000 FRW', '400,000 USD', '210,000,000 FRW',
       '230,000 USD', '140,000,000 FRW', '9,00

In [64]:
# Grouping and Aggregation

print("\n--- Average Price by Location ---")
print(df.groupby('Location')['Price_USD'].mean())

print("\n--- Count of Properties by Location ---")
print(df.groupby('Location').size())

# Price Range
price_range = df['Price_USD'].max() - df['Price_USD'].min()
print(f"\n--- Price Range: {price_range} USD ---")

# Location with the most listings
most_listings_location = df['Location'].value_counts().idxmax()
print(f"\n--- Location with the Most Listings: {most_listings_location} ---")


--- Average Price by Location ---
Location
                               1003.723951
Gacuriro                       6010.501735
Gikondo                         500.000000
Gikondo, Rebero                 500.000000
Gisenyi                       84742.080000
Gishushu                        916.784364
Gisozi                          670.274790
Gisozi at gaposho estate        800.000000
Kabeza                          615.721467
Kabuga                        98865.760000
Kacyiru                       15830.168400
Kacyiru.                        800.000000
Kagarama                        909.024262
Kagugu                          753.500500
Kanombe                         400.000000
Kibagabaga                    14839.130830
Kicukiro                       1000.000000
Kicukiro kagarama              1500.000000
Kicukiro niboyi             1000000.000000
Kicukiro,kagarama               500.000000
Kigali, Kibagabaga             1400.000000
Kigali, kabeza                  600.000000
Kigarama  

In [2]:
# # Distribution of Prices
# plt.figure(figsize=(8, 6))
# sns.histplot(df['Price_USD'], bins=10, kde=True)
# plt.title('Distribution of Property Prices')
# plt.xlabel('Price (USD)')
# plt.ylabel('Frequency')
# plt.show()

In [1]:
# # Average Price by Location (Bar Plot)
# average_price_by_location = df.groupby('Location')['Price_USD'].mean().sort_values(ascending=False)
# plt.figure(figsize=(10, 6))
# average_price_by_location.plot(kind='bar')
# plt.title('Average Price by Location')
# plt.xlabel('Location')
# plt.ylabel('Average Price (USD)')
# plt.xticks(rotation=45, ha='right')
# plt.tight_layout()
# plt.show()